# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Будем анализировать возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Дополнительные условия:**

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

# Описание данных

- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

## Импорт библиотек

Импортируем необходимые библиотеки отдельным блоком

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

Также введем отдельную константу для гиперпараметра `random_state`. Была такая рекомендация :-)

In [2]:
RANDOM_STATE = 12345

## Загрузка и подготовка данных

Загрузим файлы и взглянем на общую информацию по датафреймам регионов:

In [3]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('geo_data_0.csv')
    data_1 = pd.read_csv('geo_data_1.csv')
    data_2 = pd.read_csv('geo_data_2.csv')
    
data_0.info()
data_1.info()
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Радует, что нет пропусков и типы данных те, что надо. Взглянем визуально на данные в датафремах:

In [4]:
display(data_0.sample(3))
display(data_1.sample(3))
display(data_0.sample(3))

,id,f0,f1,f2,product
23313,6lg3M,-0.521772,0.877681,0.565540,110.619634
72935,R5ChZ,1.349718,-0.546640,0.775271,52.240115
22318,tEeaN,0.176823,-0.189074,2.909108,59.641867


,id,f0,f1,f2,product
39269,Mg6WU,12.380863,-4.128010,4.001981,107.813044
97861,0hhdU,4.168716,-3.858393,1.009855,30.132364
6181,DoTTp,-5.348585,-1.921888,0.992120,30.132364


,id,f0,f1,f2,product
56826,oljKF,-0.461230,0.937029,6.085808,83.219217
40958,EWJEK,0.817402,0.385538,5.428183,64.761759
7912,kIxvH,1.636079,-0.549042,-0.043468,126.374565


Данные выглядят красиво, конечно. Но ничего, кроме `product` непонятно... 

Проверим каждый датафрейм на наличие явных дубликатов:

In [5]:
data_0[data_0.duplicated() == True]['id'].count()

0

In [6]:
data_1[data_1.duplicated() == True]['id'].count()

0

In [7]:
data_2[data_2.duplicated() == True]['id'].count()

0

Прекрасно, одинаковых объектов у нас нет. Но интересно, есть ли одинаковые скважины в датафреймах? Проверим дубликаты в столбце `id`:

In [8]:
data_0[data_0['id'].duplicated() == True]['id'].count()

10

Имеем десять повторяющихся `id`. Это одна сотая процента данных. Посмотрим визуально на объекты с повторяющимися `id`:

In [9]:
display(data_0[data_0['id'].duplicated(keep=False) == True].sort_values('id'))

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


Каких-то закономерностей не видно. Предполагаю, что это данные по скважине за разный период или ошибка при формировании базы данных. Думаю, что для наших целей можно дубликаты оставить и считать их разными скважинами. 

Проверим аналогично остальные два датафрейма:

In [10]:
print(data_1[data_1['id'].duplicated() == True]['id'].count())
print(data_2[data_2['id'].duplicated() == True]['id'].count())

4
4


Всего четыре дубликата по `id` в обоих датафреймах. Капля в море. Глянем визуально:

In [11]:
display(data_1[data_1['id'].duplicated(keep=False) == True].sort_values('id'))
display(data_2[data_2['id'].duplicated(keep=False) == True].sort_values('id'))

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


Ситуация аналогичная "нулевому" региону. Будем считать эти скважины разными и оставим в датафрейме.

### Итоги знакомства с данными

Отлично подготовленные данные без пропусков, без одинаковых объектов и с минимумом дубликатов в столбце `id` (в сумме на три региона оказалось 18, что пренебрежимо мало по сравнению с 300 000 объектов в трех регионах). По моему мнению, предобработка не требуется. 

## Обучение и проверка модели

Перед нами стоит задача предсказания числа (а не классификации), поэтому будем использовать модель линейной регрессии. Обучим и проверим модель для каждого региона. Для этого напишем функцию:

In [12]:
def data_ml(data):
    features = data.drop(['id', 'product'], axis=1) # формируем датафрейм с признаками
    target_region = data['product'] # целевой признак
    
    # формируем обучающую и валидационную выборку
    features_train, features_valid, target_train, target_valid = train_test_split(features, target_region,
                                                                                  test_size=.25, random_state=RANDOM_STATE)
    model = LinearRegression() # создаём линейную регрессию
    model.fit(features_train, target_train) # обучаем модель

    prediction_valid = model.predict(features_valid) # предсказываем на валидационнной выборке
    prediction_region = model.predict(features) # выполняем предсказания целиком по региону для будущих действий
    rmse = mean_squared_error(target_valid, prediction_valid, squared=False) # считаем rmse
    predicted_mean_stock = prediction_valid.mean() # считаем средний запас по региону
    
    return model, prediction_valid, target_valid, prediction_region, target_region, rmse, predicted_mean_stock

Применим функцию к датафреймам и сформируем табличку с результатами:

In [13]:
model_0, prediction_valid_0, target_valid_0, prediction_region_0, target_region_0, rmse_0, predicted_mean_stock_0 = data_ml(data_0)
model_1, prediction_valid_1, target_valid_1, prediction_region_1, target_region_1, rmse_1, predicted_mean_stock_1 = data_ml(data_1)
model_2, prediction_valid_2, target_valid_2, prediction_region_2, target_region_2, rmse_2, predicted_mean_stock_2 = data_ml(data_2)

models_conclusion = pd.DataFrame({'Регион' : [0, 1, 2],
                    'RMSE' : [rmse_0, rmse_1, rmse_2],
                    'Средний запас (пред), тыс. бар.' : [predicted_mean_stock_0, predicted_mean_stock_1, predicted_mean_stock_2]})
display(models_conclusion)

,Регион,RMSE,"Средний запас (пред), тыс. бар."
0,0,37.579422,92.592568
1,1,0.893099,68.728547
2,2,40.029709,94.965046


### Итоги обучения и проверки модели

Как видим по среднему значению предсказанных запасов регионы "0" и "2" в явных лидерах, при этом среднеквадратическая ошибка по этим регионам составляет около 40% от среднего запаса.

Регион "1" менее богат на ресурсы, но среднеквадратическая ошибка невероятно мала. Похоже, данные не очень-то реальные :-)

## Подготовка к расчёту прибыли

Посчитаем минимальный запас на скважину для безубыточной добычи:

In [14]:
# создаём константы
NUM_WELLS = 500 # количество исследуемых точек
NUM_BEST_WELLS = 200 # количество скважин для разработки
TH_BARR_INCOME = 450000 # доход с тысячи баррелей
REGION_BUDGET = 10000000000 # бюджет на разработку региона

mean_well_budget = REGION_BUDGET / NUM_BEST_WELLS # считаем средний бюджет на скважину
min_mean_well_stock = mean_well_budget / TH_BARR_INCOME # считаем минимальный объём сырья, который покроет бюджет на разработку

print(f'Минимальный запас на скважину для безубыточной добычи: {min_mean_well_stock} тыс. бар')

Минимальный запас на скважину для безубыточной добычи: 111.11111111111111 тыс. бар


### Функция расчёта прибыли

Напишем функцию, на вход которой передаём реальные запасы, предсказанные запасы и количество лучших скважин:

In [15]:
def region_profit(target_stock, predicted_stock, num_best_wells):
    # сортируем предсказания в порядке убывания
    predicted_stock_sorted = predicted_stock.sort_values(ascending=False)
    # выбираем самые перспективные
    selected_wells = target_stock[predicted_stock_sorted.index].head(num_best_wells)
    # считаем суммарный запас сыръя в лучших скважинах
    sum_stock = selected_wells.sum()
    # считаем прибыль с региона в миллионах рублей для удобства восприятия
    region_profit = (sum_stock * TH_BARR_INCOME - REGION_BUDGET) / 1000000
    return region_profit

### Итоги подготовки к расчёту прибыли

Как видим, средний запас скважины по всем регионам (**69 тыс., 93 тыс., 95 тыс. баррелей**) значимо ниже расчётного минимального запаса (**111 тыс. баррелей**). Поэтому крайне важно выбирать лучшие скважины по запасу сырья.

Также подготовили функцию для расчёта прибыли с региона.

## Расчёт прибыли и рисков 

Напишем функцию для оценки прибыли по региону с помощью бутстреп. На вход функция будет получать целевой признак и предсказания модели по региону:

In [16]:
def bootstrap_profit(target, prediction):
    state = np.random.RandomState(12345) # переменная для смены "псевдослучайности"

    profits = [] # список для получившихся значений прибыли

    # преобразуем данные для корректной работы
    target = target.reset_index(drop=True) 
    prediction = pd.Series(prediction)
    
    for i in range(1000):
        # формируем случайную выборку из реальных запасов в количестве исследуемых точек
        target_wells_stock = target.sample(NUM_WELLS, replace=True, random_state=state)
        # по индексам выбранных точек, выбираем соответствующие этим точкам предсказания
        predicted_stock_subsample = prediction[target_wells_stock.index]
        # считаем прибыль по региону и добавляем результат в список
        profits.append(region_profit(target_wells_stock, predicted_stock_subsample, NUM_BEST_WELLS))

    profits = pd.Series(profits)
    lower = round(profits.quantile(.025), 2)
    upper = round(profits.quantile(.975), 2)
    mean = round(profits.mean(), 2)

    return profits, mean, lower, upper

Создадим функцию для расчёта рисков. Она будет "пробегаться" по списку прибылей в результате применения бутстрепа, считать количество отрицательных значений и считать процент таких значений, что и будет процентом риска.

In [17]:
def loss_risk(profits):
    i = 0
    for profit in profits:
        if profit < 0:
            i += 1
    
    loss_risk = (i / len(profits)) * 100
    return loss_risk

Применим функции для каждого из регионов и сформируем таблицу с результатами:

In [18]:
profits_0, mean_0, lower_0, upper_0 = bootstrap_profit(target_region_0, prediction_region_0)
profits_1, mean_1, lower_1, upper_1 = bootstrap_profit(target_region_1, prediction_region_1)
profits_2, mean_2, lower_2, upper_2 = bootstrap_profit(target_region_2, prediction_region_2)

loss_risk_0 = loss_risk(profits_0)
loss_risk_1 = loss_risk(profits_1)
loss_risk_2 = loss_risk(profits_2)

profit_conclusion = pd.DataFrame({'Регион' : [0, 1, 2],
                                 'Средняя прибыль с региона, млн' : [mean_0, mean_1, mean_2],
                                 '2.5%-квантиль, млн' : [lower_0, lower_1, lower_2],
                                 '97.5%-квантиль, млн' : [upper_0, upper_1, upper_2],
                                 'Риск убытков, %' : [loss_risk_0, loss_risk_1, loss_risk_2]})

display(profit_conclusion)

,Регион,"Средняя прибыль с региона, млн","2.5%-квантиль, млн","97.5%-квантиль, млн","Риск убытков, %"
0,0,431.09,-70.54,979.02,5.1
1,1,462.96,55.99,845.84,1.3
2,2,380.55,-177.06,896.99,9.1


### Итоги расчёта прибыли и рисков

- Наиболее прибыльный регион - номер 1 (**Средняя прибыль - 463 млн**)
- Регион с максимально возможным убытком по итогам бутстрепа - номер 2 (**2.5%-квантиль: -177 млн.**)
- Регион с максимальным "джек-потом" по прибыли по итогам бутстрепа - номер 0 (**97.5%-квантиль: 979 млн.**)
- Регион с минимальным риском - номер 1 (**1,3%**)
- Регион с максимальным риском убытков - номер 2 (**9,1%**)

## Общий итог

Мы получили отлично подготовленные данные без пропусков, без одинаковых объектов и с минимумом дубликатов в столбце id (в сумме на три региона оказалось 18, что пренебрежимо мало по сравнению с 300 000 объектов в трех регионах). По моему мнению, предобработка не потребовалась.

Затем обучили модели по трем регионам и определили средний предсказанный запас сырья скважины, а также среднеквадратическую ошибку предсказания:

In [19]:
display(models_conclusion)

,Регион,RMSE,"Средний запас (пред), тыс. бар."
0,0,37.579422,92.592568
1,1,0.893099,68.728547
2,2,40.029709,94.965046


По среднему значению предсказанных запасов регионы "0" и "2" оказались в явных лидерах, при этом среднеквадратическая ошибка по этим регионам составляет около 40% от среднего предсказанного запаса.

Регион "1" менее богат на ресурсы, но среднеквадратическая ошибка невероятно мала.

После этого посчитали средний запас сырья в скважине для безубыточной добычи - **111 тыс. баррелей**.
Оказалось, что средний запас скважины по всем регионам (**69 тыс., 93 тыс., 95 тыс. баррелей**) значимо ниже расчётного минимального запаса. Поэтому крайне важно выбирать лучшие скважины по запасу сырья.

Также подготовили функцию для расчёта прибыли с региона.

Дальнейшие расчёты с применением бутстрепа показали следующие результаты:

In [20]:
display(profit_conclusion)

,Регион,"Средняя прибыль с региона, млн","2.5%-квантиль, млн","97.5%-квантиль, млн","Риск убытков, %"
0,0,431.09,-70.54,979.02,5.1
1,1,462.96,55.99,845.84,1.3
2,2,380.55,-177.06,896.99,9.1


**Таким образом наименее рискованный и прибыльный регион оказался регион под номером "1". Его бы и рекомендовал к разработке.**

Не сказал бы изначально, что такой результат будет т.к. средний запас сырья в этом регионе ниже всех, но модель с великолепной точностью отбирает лучшие скважины, что и предопределяет успех.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован